<a href="https://colab.research.google.com/github/puraminy/mini_proj2/blob/master/BAM3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class BAM(object):
    def __init__(self, data):
        self.pairs = data
        m1 = len(self.pairs[0][1])
        m2 = len(self.pairs[0][0])
        # create weight matrix
        self.W = np.zeros((m2,m1))
        for p in self.pairs:
          X = p[0]
          Y = p[1]
          self.W += np.outer(X, Y)

    def get_assoc(self, A):
        A = np.asarray(A)
        y=np.dot(A.T,self.W)
        y[y<0]=-1
        y[y>=0]=1
        return y

  

import matplotlib.pyplot as plt

def draw_img(ax, X):
  ax.imshow(np.reshape(X,[5,3]))
  
def test_noise (inp, te_n, n_n): #inp:input vector te_n: number of noisy samples for test. n_n: number of noisy elements
  test = []
  for i in range(te_n):
    rnd = random.sample(range(len(inp)), n_n)
    noise = np.asarray(inp)
    for i in rnd:
      noise[i] = -1*noise[i]
    test.append(noise)

  test=np.asarray(test)
  test = np.unique(test,axis=0)

  return test


In [47]:
import numpy as np
import random

A = [-1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1]
tA = [-1, -1,-1] 
B = [1, 1, -1, 1, -1, 1, 1, 1, -1, 1, -1, 1, 1, 1, -1]
tB = [-1, -1, 1]
C = [-1, 1, 1, 1, -1, -1, 1, -1, -1, 1, -1, -1, -1, 1, 1]
tC = [-1,1,-1] 
D = [1, 1, -1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1]
tD = [-1,1,1]
E = [1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, 1, 1]
tE = [1,-1,-1]
F =[1, 1, 1, 1, -1, -1, 1, 1, -1, 1, -1, -1, 1, -1, -1]
tF = [1, -1, 1]
G = [-1, 1, 1, 1, -1, -1, 1, -1, 1, 1, -1, 1, -1, 1, 1]
tG = [1, 1, -1]
H = [1, -1, 1, 1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1]
tH = [1, 1, 1]

# tA = [-1, 1] 
# tC = [1, 1] 
letters ={"A" : A,"B" : B, "C" : C,"D":D, "E":E,"F":F,"G":G,"H":H}
targets ={"A" : tA,"B": tB, "C" : tC, "D" : tD, 
           "E" : tE,"F": tF, "G" : tG, "H" : tH}

# fig, axes = plt.subplots(2, 4)
# plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.5)
# for i in range(4):
#   draw_img(axes[0,i], X[i])
#   xlabel = Y[i]; axes[0,i].set_xlabel(xlabel)
#   draw_img(axes[1,i], X[i+4]) 
#   xlabel = Y[i+4]; axes[1,i].set_xlabel(xlabel)

ds=[]

ds.append(dict((k, letters[k]) for k in ('A','C')))
ds.append(dict((k, letters[k]) for k in ('A','B')))
ds.append(dict((k, letters[k]) for k in ('E','D')))
ds.append(dict((k, letters[k]) for k in ('D','C')))
ds.append(dict((k, letters[k]) for k in ('A','B','C')))
ds.append(dict((k, letters[k]) for k in ('F','C','H')))
ds.append(dict((k, letters[k]) for k in ('A','B','C','D','E')))
ds.append(dict((k, letters[k]) for k in ('A','B','C','D','E','F')))
ds.append(dict((k, letters[k]) for k in ('A','B','C','D','E','F','G','H')))


for data in ds:
  print("==========")
  print(data.keys())
  pairs = []

  for key,value in data.items():
    y = targets[key]
    pairs.append([value, y])

  bam = BAM(pairs)

  for key,letter in data.items():
    target = targets[key]
    test_data=test_noise(letter, 100, 3)
    success = 0
    for i,noisy_data in enumerate(test_data):
      predicted = bam.get_assoc(noisy_data)
      if np.array_equal(target, predicted):
        success +=1

    print(f"Accuracy of {key}= {success}")


    


dict_keys(['A', 'C'])
Accuracy of A= 90
Accuracy of C= 85
dict_keys(['A', 'B'])
Accuracy of A= 75
Accuracy of B= 93
dict_keys(['E', 'D'])
Accuracy of E= 86
Accuracy of D= 86
dict_keys(['D', 'C'])
Accuracy of D= 85
Accuracy of C= 87
dict_keys(['A', 'B', 'C'])
Accuracy of A= 76
Accuracy of B= 77
Accuracy of C= 77
dict_keys(['F', 'C', 'H'])
Accuracy of F= 61
Accuracy of C= 75
Accuracy of H= 85
dict_keys(['A', 'B', 'C', 'D', 'E'])
Accuracy of A= 54
Accuracy of B= 73
Accuracy of C= 42
Accuracy of D= 36
Accuracy of E= 26
dict_keys(['A', 'B', 'C', 'D', 'E', 'F'])
Accuracy of A= 47
Accuracy of B= 80
Accuracy of C= 31
Accuracy of D= 40
Accuracy of E= 44
Accuracy of F= 47
dict_keys(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H'])
Accuracy of A= 10
Accuracy of B= 70
Accuracy of C= 21
Accuracy of D= 42
Accuracy of E= 35
Accuracy of F= 61
Accuracy of G= 47
Accuracy of H= 21
